<a href="https://colab.research.google.com/github/yohanesnuwara/geostatistics/blob/main/project_notebooks/volve_sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook tries to demonstrate how to transform a production Excel spreadsheet of Volve field into an SQL database, a format that data scientists mostly work on. This uses SQLite3 in Python.

The spreadsheet contains 2 sheets, "Daily Production Data" and "Monthly Production Data". The plan is to make these 2 separated sheets into 1 database (relational database). 

In [ ]:
!sudo apt-get install -y sqlite3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sqlite3 is already the newest version (3.22.0-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sqlite3

In [ ]:
!git clone https://github.com/yohanesnuwara/volve-machine-learning

Cloning into 'volve-machine-learning'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 118
Receiving objects: 100% (121/121), 17.98 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (51/51), done.


We need to access to our production dataset in Excel format. A filepath has been defined, it is from a GitHub repository. Then, using Pandas, read the Excel file. We do twice, each for the two sheets `Daily Production Data` & `Monthly Production Data`. Please mind to skip the 2nd row of the 2nd sheet, passing `skiprows=[1]`. 

In [ ]:
# Excel file path from GitHub repository
filepath = 'https://github.com/yohanesnuwara/volve-machine-learning/raw/main/Volve%20production%20data.xlsx'

# Read the first sheet
daily_df = pd.read_excel(filepath, sheet_name='Daily Production Data')

# Read the second sheet. Mind to skip the 2nd row. 
month_df = pd.read_excel(filepath, sheet_name='Monthly Production Data', skiprows=[1])

# Overview the two dataframes
month_df.head(10)

,Wellbore name,NPDCode,Year,Month,On Stream,Oil,Gas,Water,GI,WI
0,15/9-F-1 C,7405,2014,4,227.50000,11142.47,1597936.65,0.00,NaN,NaN
1,15/9-F-1 C,7405,2014,5,733.83334,24901.95,3496229.65,783.48,NaN,NaN
2,15/9-F-1 C,7405,2014,6,705.91666,19617.76,2886661.69,2068.48,NaN,NaN
3,15/9-F-1 C,7405,2014,7,742.41666,15085.68,2249365.75,6243.98,NaN,NaN
4,15/9-F-1 C,7405,2014,8,432.99166,6970.43,1048190.80,4529.75,NaN,NaN
5,15/9-F-1 C,7405,2014,9,630.30000,9168.43,1414099.99,8317.59,NaN,NaN
6,15/9-F-1 C,7405,2014,10,745.00000,9468.06,1462063.99,10364.87,NaN,NaN
7,15/9-F-1 C,7405,2014,11,579.77500,6710.33,1044188.30,7234.24,NaN,NaN
8,15/9-F-1 C,7405,2014,12,27.50000,120.29,25857.08,183.44,NaN,NaN
9,15/9-F-1 C,7405,2015,1,479.91667,10875.53,1604934.60,6850.80,NaN,NaN


If we compare both dataframes, we see some column names that shares the same information, have different names.

In [ ]:
# Change column names of monthly data
month_df = month_df.rename(columns={'Wellbore name': 'NPD_WELL_BORE_NAME',
                                  'NPDCode': 'NPD_WELL_BORE_CODE',
                                  'On Stream': 'ON_STREAM_HRS',
                                  })
# Overview the dataframe
month_df.head(10)

,NPD_WELL_BORE_NAME,NPD_WELL_BORE_CODE,Year,Month,ON_STREAM_HRS,Oil,Gas,Water,GI,WI
0,15/9-F-1 C,7405,2014,4,227.50000,11142.47,1597936.65,0.00,NaN,NaN
1,15/9-F-1 C,7405,2014,5,733.83334,24901.95,3496229.65,783.48,NaN,NaN
2,15/9-F-1 C,7405,2014,6,705.91666,19617.76,2886661.69,2068.48,NaN,NaN
3,15/9-F-1 C,7405,2014,7,742.41666,15085.68,2249365.75,6243.98,NaN,NaN
4,15/9-F-1 C,7405,2014,8,432.99166,6970.43,1048190.80,4529.75,NaN,NaN
5,15/9-F-1 C,7405,2014,9,630.30000,9168.43,1414099.99,8317.59,NaN,NaN
6,15/9-F-1 C,7405,2014,10,745.00000,9468.06,1462063.99,10364.87,NaN,NaN
7,15/9-F-1 C,7405,2014,11,579.77500,6710.33,1044188.30,7234.24,NaN,NaN
8,15/9-F-1 C,7405,2014,12,27.50000,120.29,25857.08,183.44,NaN,NaN
9,15/9-F-1 C,7405,2015,1,479.91667,10875.53,1604934.60,6850.80,NaN,NaN


In [ ]:
daily_df.columns

Index(['DATEPRD', 'WELL_BORE_CODE', 'NPD_WELL_BORE_CODE', 'NPD_WELL_BORE_NAME',
       'NPD_FIELD_CODE', 'NPD_FIELD_NAME', 'NPD_FACILITY_CODE',
       'NPD_FACILITY_NAME', 'ON_STREAM_HRS', 'AVG_DOWNHOLE_PRESSURE',
       'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING', 'AVG_ANNULUS_PRESS',
       'AVG_CHOKE_SIZE_P', 'AVG_CHOKE_UOM', 'AVG_WHP_P', 'AVG_WHT_P',
       'DP_CHOKE_SIZE', 'BORE_OIL_VOL', 'BORE_GAS_VOL', 'BORE_WAT_VOL',
       'BORE_WI_VOL', 'FLOW_KIND', 'WELL_TYPE'],
      dtype='object')

In [ ]:
def write_sql(df, datatype_array, table_name, script_name):
  """
  Write an SQL script that writes a table

  Input:

  df: DataFrame to be transformed into SQL table
  datatype_array: array of SQL datatypes with respect to each column name
  table_name: name of SQL table that you want to produce
  script_name: name of SQL script that you will feed to your database

  Output:

  SQL script with extension .sql
  Also it prints the whole script 
  """

  import pandas as pd

  # List the unique column names into array
  columns = df.columns

  # Write SQL script
  sql_file = 'CREATE TABLE IF NOT EXISTS {} ( \n'.format(table_name)
  for i in range(len(columns)):
    if i == len(columns)-1: # last column name without comma
      sql_file += ' {} {} \n'.format(columns[i], datatype[i])
    else: # give commas
      sql_file += ' {} {}, \n'.format(columns[i], datatype[i])
  sql_file += ');'

  # Print the SQL script
  print(sql_file)

  # Write into an SQL file
  inc_out = open(script_name + '.sql', 'w')
  inc_out.write(sql_file)
  inc_out.close() 

In [ ]:
month_df.columns

Index(['NPD_WELL_BORE_NAME', 'NPD_WELL_BORE_CODE', 'Year', 'Month',
       'ON_STREAM_HRS', 'Oil', 'Gas', 'Water', 'GI', 'WI'],
      dtype='object')

In [ ]:
datatype_daily = ['TEXT', 'INTEGER', 'INTEGER', 'TEXT', 
                  'INTEGER', 'TEXT', 'INTEGER', 
                  'TEXT', 'REAL', 'REAL', 'REAL', 
                  'REAL', 'REAL', 'REAL', 'REAL', 
                  'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 
                  'REAL', 'REAL', 'REAL', 'TEXT', 'TEXT']

datatype_month = ['TEXT', 'INTEGER']

In [ ]:
# SQL script for the "daily" dataframe
datatype = ['TEXT', 'INTEGER', 'INTEGER', 'TEXT', 
            'INTEGER', 'TEXT', 'INTEGER', 
            'TEXT', 'REAL', 'REAL', 'REAL', 
            'REAL', 'REAL', 'REAL', 'REAL', 
            'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 
            'REAL', 'REAL', 'REAL', 'TEXT', 'TEXT']
write_sql(daily_df, datatype, 'daily', 'create_table_daily')

# SQL script for the "daily" dataframe
# write_sql(month_df, datatype, 'daily', 'create_table_daily')

In [ ]:
db_conn = sqlite3.connect('Volve production data.db')

c = db_conn.cursor()

In [ ]:
%%writefile create_table.sql
CREATE TABLE IF NOT EXISTS daily (
 DATEPRD TEXT,
 WELL_BORE_CODE INTEGER,
 NPD_WELL_BORE_CODE INTEGER,
 NPD_WELL_BORE_NAME TEXT,
 NPD_FIELD_CODE INTEGER,
 NPD_FIELD_NAME TEXT,
 NPD_FACILITY_CODE INTEGER,
 NPD_FACILITY_NAME TEXT,
 ON_STREAM_HRS REAL,
 AVG_DOWNHOLE_PRESSURE REAL,
 AVG_DOWNHOLE_TEMPERATURE REAL,
 AVG_DP_TUBING REAL,
 AVG_ANNULUS_PRESS REAL,
 AVG_CHOKE_SIZE_P REAL,
 AVG_CHOKE_UOM REAL,
 AVG_WHP_P REAL,
 AVG_WHT_P REAL,
 DP_CHOKE_SIZE REAL,
 BORE_OIL_VOL REAL,
 BORE_GAS_VOL REAL,
 BORE_WAT_VOL REAL,
 BORE_WI_VOL REAL,
 FLOW_KIND REAL,
 WELL_TYPE TEXT
);

Overwriting create_table.sql


In [ ]:
print(datatype)

['TEXT', 'INTEGER', 'INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'TEXT', 'TEXT']


In [ ]:
!sqlite3 'Volve production data.db' < create_table.sql

In [ ]:
!sqlite3 'Volve production data.db' < create_table_daily.sql

In [ ]:
!sqlite3 c < create_table.sql

In [ ]:
c.execute(
    """
    CREATE TABLE IF NOT EXISTS monthly (
      NPD_WELL_BORE_NAME TEXT,
      NPD_WELL_BORE_CODE INTEGER,
      Year INTEGER,
      Month INTEGER,
      ON_STREAM_HRS REAL,
      Oil REAL,
      Gas REAL,
      Water REAL,
      GI REAL,
      WI REAL
    );
    """
)

In [ ]:
month_df.to_sql('monthly', db_conn, if_exists='append', index=False)

In [ ]:
daily_df.to_sql('daily', db_conn, if_exists='append', index=False)

In [ ]:
pd.read_sql("SELECT * FROM daily LIMIT 10", db_conn)

,DATEPRD,WELL_BORE_CODE,NPD_WELL_BORE_CODE,NPD_WELL_BORE_NAME,NPD_FIELD_CODE,NPD_FIELD_NAME,NPD_FACILITY_CODE,NPD_FACILITY_NAME,ON_STREAM_HRS,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_CHOKE_UOM,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,BORE_WI_VOL,FLOW_KIND,WELL_TYPE
0,2014-04-07 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,0.00000,0.00000,0.00000,0.0,0.000000,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,None,production,WI
1,2014-04-08 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,NaN,NaN,0.0,1.003059,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,None,production,OP
2,2014-04-09 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,NaN,NaN,0.0,0.979008,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,None,production,OP
3,2014-04-10 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,NaN,NaN,NaN,0.0,0.545759,%,0.00000,0.00000,0.00000,0.0,0.0,0.0,None,production,OP
4,2014-04-11 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,310.37614,96.87589,277.27826,0.0,1.215987,%,33.09788,10.47992,33.07195,0.0,0.0,0.0,None,production,OP
5,2014-04-12 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,303.50078,96.92339,281.44744,0.0,3.087015,%,22.05334,8.70429,22.05334,0.0,0.0,0.0,None,production,OP
6,2014-04-13 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,303.53481,96.95885,276.03200,0.0,1.962365,%,27.50281,9.42315,16.16326,0.0,0.0,0.0,None,production,OP
7,2014-04-14 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,303.78228,96.96873,282.78676,0.0,0.000000,%,20.99552,8.13137,20.73712,0.0,0.0,0.0,None,production,OP
8,2014-04-15 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,303.85821,97.02136,289.94067,0.0,31.141856,%,13.91754,8.49833,12.18153,0.0,0.0,0.0,None,production,OP
9,2014-04-16 00:00:00,NO 15/9-F-1 C,7405,15/9-F-1 C,3420717,VOLVE,369304,MÆRSK INSPIRER,0.0,303.79187,97.06569,299.67193,0.0,0.000000,%,4.11994,8.82124,1.49020,0.0,0.0,0.0,None,production,OP
